In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar100
import time
import numpy as np
import cv2 as cv
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [3]:
images_train = []
images_test = []

for i in range(0, 50000):
    image = x_train[i, :, :, :]  # Original image
    image = cv.resize(image, (128, 128))  # Resize using OpenCV
    images_train.append(image)

for i in range(0, 10000):
    image = x_test[i, :, :, :]  # Original image
    image = cv.resize(image, (128, 128))  # Resize using OpenCV
    images_test.append(image)

In [4]:
hog_features_train = []
hog_features_test = []

hog = cv.HOGDescriptor(_winSize=(128, 128), _blockSize=(16, 16), _blockStride=(8, 8), _cellSize=(8, 8), _nbins=9)


for image in images_train:
    gray_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    features = hog.compute(gray_image)
    hog_features_train.append(features)

for image in images_test:
    gray_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    features = hog.compute(gray_image)
    hog_features_test.append(features)


In [5]:
hog_features_train = np.array(hog_features_train)
hog_features_test = np.array(hog_features_test)

hog=np.concatenate((hog_features_train, hog_features_test), axis=0)

mean = np.mean(hog, axis=0)
std = np.std(hog, axis=0)

hog_features_test = (hog_features_test - mean) / std
hog_features_train = (hog_features_train - mean) / std


y_train = y_train.flatten()  # Flatten the target labels
y_test = y_test.flatten()

In [ ]:


# Train SVM classifier
model = SVC(kernel='linear', C=1)  # Use a linear kernel
model.fit(hog_features_train, y_train)

# Predict on test data
y_pred = model.predict(hog_features_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of SVM classifier: {accuracy * 100:.2f}%")

Accuracy of SVM classifier: 22.31%


In [8]:
import joblib

# Save the model to a file
joblib.dump(model, 'hog_svm.pkl')

['hog_svm.pkl']